In [10]:
import numpy as np
from scipy import io as sio
from matplotlib import pyplot as plt

In [11]:
import typhon as tp

In [12]:
# Set input frequency values.

# Case 1
# f_grid = np.array([18.700e9, 22.235e9, 37.000e9, 40.000e9, 50.300e9, 52.300e9, 53.600e9, 54.550e9, 55.750e9, 57.000e9, 58.400e9, 59.800e9])

# Case 2
f_grid = np.array([ 22.234e9, 22.5e9, 23.034e9, 23.834e9, 
                    25e9, 26.234e9, 28e9, 30e9, 
                    51.248e9, 51.76e9, 52.28e9, 52.804e9, 
                    53.336e9, 53.848e9, 54.4e9, 54.94e9, 
                    55.5e9, 56.02e9, 56.66e9, 57.288e9, 57.964e9, 58.8e9 ])
                   
tp.arts.xml.save(f_grid, './ClearSky_1D_f_grid.xml')

/home/user/anaconda3/lib/python3.6/site-packages/typhon/arts/utils.py:32: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  if np.issubdtype(var.dtype, np.complex):
/home/user/anaconda3/lib/python3.6/site-packages/typhon/arts/xml/write.py:185: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  if np.issubdtype(var.dtype, np.complex):
/home/user/anaconda3/lib/python3.6/site-packages/typhon/arts/xml/write.py:197: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  if np.issubdtype(var.dtype, np.complex):


In [13]:
# Set sensor viewing angles and position.

# Case 1
sensor_los = np.array([[171.030788, 90]]) # Looking down at an angle.
sensor_pos = np.array([[5334.194]]) # Sensor altitude
sensor_gloc = np.array([34.511312, 127.224312]) # Sensor geolocation. 

# Case 2
# sensor_los = np.array([[0, 0]]) # Looking straight up
# sensor_pos = np.array([[0]]) # Sensor geolocation does not matter since it is 1-D calculation. 
# sensor_gloc = np.array([34.763892, 127.212426]) # Bosung, Korea

tp.arts.xml.save(sensor_los, './ClearSky_1D_sensor_los.xml')
tp.arts.xml.save(sensor_pos, './ClearSky_1D_sensor_pos.xml')

/home/user/anaconda3/lib/python3.6/site-packages/typhon/arts/utils.py:32: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  if np.issubdtype(var.dtype, np.complex):
/home/user/anaconda3/lib/python3.6/site-packages/typhon/arts/xml/write.py:185: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  if np.issubdtype(var.dtype, np.complex):
/home/user/anaconda3/lib/python3.6/site-packages/typhon/arts/xml/write.py:214: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  if np.issubdtype(var.dtype, np.complex):


In [14]:
# Load surface and atmosphere datasets from netCDF input files. 
surface_dataset = sio.netcdf.netcdf_file('./l015v070erlounish000.2017093000.rec.nc',mmap=False)
atmosphere_dataset = sio.netcdf.netcdf_file('./l015v070erlopresh000.2017093000.rec.nc',mmap=False)

In [15]:
# Find the geolocation of the pixel that is closest to the geolocation of the sensor's line-of-sight (LOS). 

In [16]:
# Assume WGS 1984 for the reference Ellipsoid.
R_eq = 6378137 # Earth's equatorial radius, in meters
iFlttn = 298.257223563 # Inverse flattening
R_polar = R_eq * (1-1/iFlttn) # Earth's polar radius
eccnty = 2/iFlttn - (iFlttn)**2 # Eccentricity of the ellipsoid. 

In [17]:
# Convert the sensor's position from polar to Cartesian coordinates.
sensor_pos_alt = sensor_pos[0][0]
sensor_pos_lat = sensor_gloc[0]
sensor_pos_lon = sensor_gloc[1]

# Fix!!! What does ARTS use for reference ellipsoid/sphere for 1-D calculations? This affects sensor_pos and z_field_raw. 
sensor_pos_cart1 = np.array([ (R_eq) * np.cos(sensor_pos_lat * np.pi/180) * np.cos(sensor_pos_lon * np.pi/180), 
    (R_eq) * np.cos(sensor_pos_lat * np.pi/180) * np.sin(sensor_pos_lon * np.pi/180), 
    (R_polar) * np.sin(sensor_pos_lat * np.pi/180) ]) # Cartesian coordinates of the sensor's position

sensor_pos_hvector = np.array([sensor_pos_cart1[0]/(R_eq**2), sensor_pos_cart1[1]/(R_eq**2), sensor_pos_cart1[2]/(R_polar**2)])
ttt = sensor_pos_alt/(sensor_pos_hvector[0]**2 + sensor_pos_hvector[1]**2 + sensor_pos_hvector[2]**2)**.5
sensor_pos_h_cart = ttt * sensor_pos_hvector

sensor_pos_cart = sensor_pos_cart1 + sensor_pos_h_cart
sensor_pos_cart

array([-3182009.45807626,  4188450.98437184,  3604567.46616321])

In [18]:
# If the sensor is on the ground looking above:
if (sensor_pos_alt == 0 and sensor_los[0][0] <= 90):
    sensor_los_gpos = sensor_pos_cart

# If the sensor is above the ground looking down: 
elif (sensor_pos_alt > 0 and sensor_los[0][0] <= 180):
    # Find the local reference vectors for the sensor's zenith and azimuth. 
    sensor_los_localZ = -np.array([sensor_pos_cart[0]/((R_eq+sensor_pos_alt)**2), 
                                   sensor_pos_cart[1]/((R_eq+sensor_pos_alt)**2), 
                                   sensor_pos_cart[2]/((R_polar+sensor_pos_alt)**2)])
    sensor_pos2NorthPole = np.array([0, 0, R_polar]) - sensor_pos_cart
    sensor_pos2North = sensor_pos2NorthPole - ((np.dot(sensor_pos2NorthPole,sensor_los_localZ) / np.dot(sensor_los_localZ,sensor_los_localZ)) * sensor_los_localZ)
    sensor_los_localX = sensor_pos2North
    sensor_los_localY = np.cross(sensor_los_localZ,sensor_los_localX)

    # Normalize the local axes for sensor_los.
    sensor_los_localZ = sensor_los_localZ / np.linalg.norm(sensor_los_localZ)
    sensor_los_localY = sensor_los_localY / np.linalg.norm(sensor_los_localY)
    sensor_los_localX = sensor_los_localX / np.linalg.norm(sensor_los_localX)

    # Calculate the sensor's LOS vector in reference to the Earth.
    sensor_los_theta = (sensor_los[0][0] - 90) * np.pi/180
    sensor_los_phi = (sensor_los[0][1]) * np.pi/180
    sensor_los_vec = np.array([np.cos(sensor_los_theta) * np.cos(sensor_los_phi), 
                                    np.cos(sensor_los_theta) * np.sin(sensor_los_phi), 
                                    np.sin(sensor_los_theta) ])
    sensor_los_vec_global = (sensor_los_vec[0]*sensor_los_localX + sensor_los_vec[1]*sensor_los_localY + sensor_los_vec[2]*sensor_los_localZ)

    # Calculate the intersection between the Earth's ellipsoid and the sensor's LOS. 
    a = (R_polar**2) * (sensor_los_vec_global[0]**2 + sensor_los_vec_global[1]**2) + (R_eq**2) * (sensor_los_vec_global[2]**2)
    b = 2 * ((R_polar**2) * (sensor_los_vec_global[0]*sensor_pos_cart[0] + sensor_los_vec_global[1]*sensor_pos_cart[1]) + (R_eq**2) * sensor_los_vec_global[2]*sensor_pos_cart[2])
    c = (R_polar**2) * (sensor_pos_cart[0]**2 + sensor_pos_cart[1]**2) + (R_eq**2) * (sensor_pos_cart[2]**2) - (R_polar**2)*(R_eq**2)
    t = np.array([(-b + (b**2 - 4*a*c)**0.5)/(2*a), (-b - (b**2 - 4*a*c)**0.5)/(2*a)])

    sensor_los_gpos_cand = sensor_pos_cart + np.matrix(t).T*np.matrix(sensor_los_vec_global)
    sensor_los_gpos_cand_dist = np.linalg.norm(sensor_pos_cart - sensor_los_gpos_cand,axis=1)
    sensor_los_gpos_cand_dist_which = sensor_los_gpos_cand_dist < np.mean(sensor_los_gpos_cand_dist)
    sensor_los_gpos = sensor_los_gpos_cand[sensor_los_gpos_cand_dist_which,:]
    sensor_los_gpos = np.array(sensor_los_gpos)[0]
    
# Throw an error if the sensor position and LOS are not properly defined. 
else:
    raise Exception('Wrong sensor LOS and/or position! \n')
    
sensor_los_gpos

array([-3180023.73166059,  4184445.43530778,  3601538.35386836])

In [19]:
# Extract variables for latitude and longitude. 
surface_lat = surface_dataset.variables['lat'][:]
surface_lon = surface_dataset.variables['lon'][:]

# Variables sizes
surface_lat_size = len(surface_lat)
surface_lon_size = len(surface_lon)

In [20]:
# Calculate the Cartesian coordinates of the surface pixels. 
surface_lat_ext = np.array(np.tile(np.matrix(surface_lat).T,(1,surface_lon_size)))
surface_lon_ext = np.array(np.tile(np.matrix(surface_lon),(surface_lat_size,1)))

surface_pixel_cart = np.array([R_eq*np.cos(surface_lat_ext*np.pi/180)*np.cos(surface_lon_ext*np.pi/180),
                               R_eq*np.cos(surface_lat_ext*np.pi/180)*np.sin(surface_lon_ext*np.pi/180),
                               R_polar*np.sin(surface_lat_ext*np.pi/180)])

In [21]:
# Find the indices of the pixel closest to the sensor_los ground location. 
surface_pixel_cart_diff = surface_pixel_cart - np.expand_dims((np.expand_dims(sensor_los_gpos,axis=1)),axis=1)
surface_pixel_cart_dist = np.zeros((surface_lat_size,surface_lon_size))

for i in range(surface_lat_size):
    for j in range(surface_lon_size):
        surface_pixel_cart_dist[i,j] = np.linalg.norm(surface_pixel_cart_diff[:,i,j])

sensor_los_indices = np.unravel_index(np.argmin(surface_pixel_cart_dist),(surface_lat_size,surface_lon_size))
sensor_los_indices

(153, 128)

In [22]:
# Load variables from the atmospheric dataset.
atm_pressure = atmosphere_dataset.variables['lev'][:]
atm_temperature = atmosphere_dataset.variables['t'][:]  
atm_geopotentialHeight = atmosphere_dataset.variables['gh'][:]  
atm_RH = atmosphere_dataset.variables['r'][:]  
# atm_RH_Ice = atmosphere_dataset.variables['param194.1.0'][:]  

In [23]:
# Load variables from the surface dataset. 
sf_pressure = surface_dataset.variables['sp'][:]
sf_temperature = surface_dataset.variables['t_2'][:]
sf_RH = surface_dataset.variables['r'][:]

# Find the surface pressure value in the sensor's LOS. 
sf_pressure_select = sf_pressure[0,sensor_los_indices[0],sensor_los_indices[1]]
sf_pressure_select

102303.87

In [24]:
# Create p_grid
pressure = np.append(np.array([sf_pressure_select]), atm_pressure)
pressure

array([102303.8671875, 100000.       ,  97500.       ,  95000.       ,
        92500.       ,  90000.       ,  87500.       ,  85000.       ,
        80000.       ,  75000.       ,  70000.       ,  65000.       ,
        60000.       ,  55000.       ,  50000.       ,  45000.       ,
        40000.       ,  35000.       ,  30000.       ,  25000.       ,
        20000.       ,  15000.       ,  10000.       ,   7000.       ,
         5000.       ])

In [26]:
sensor_gloc[1]

127.224312

In [ ]:
tp.geodesy.ellipsoidmodels()['WGS84']